In [1]:
import sys
sys.path.insert(0, '..')

from scipy import signal
import numpy as np

import py_utils as pu
from py_utils import visualization as vz

# np.random.seed(0)

"""Generate Frames"""
preamble = pu.framing.zadoff_chu(31)
preamble = pu.modulation.modulate_qpsk(pu.framing.pn(31*2))
payload = pu.modulation.modulate_qpsk(np.random.randint(2, size=10))
len_payload = 5
frames = pu.framing.to_frames(preamble, payload, len_payload)

"""Apply channel"""
sig = np.concatenate((np.zeros(10), frames[0], np.zeros(100)))
# sig = pu.channel.apply_fto(frames, 200)
# sig = pu.channel.apply_awgn(sig, 10)
# sig = pu.channel.apply_sto(sig, 0.2)
pct = 0.041
w_off = pct * 2 * np.pi / 100
sig = pu.channel.apply_cfo(sig, w_offset=w_off)

"""Run frame detector"""
t = 0.6  # threshold
fd = pu.framing.AcquisitionFrameDetector(preamble, len(preamble), t)
res = fd.process(sig)[0]
first_detected_frame = res.frame
vz.plot_signal(first_detected_frame, sig[10:10 + len(preamble)])
print(f"Real: {w_off}\nDetector: {res.cfo}")
print(f"CFO vector: {fd.cfo_vector}")

"""Plot results"""
# t_vec = t * np.ones_like(sig)
# vz.plot_signal(cfd.debug[0], sig*0.3, t_vec, n_samps=len(sig))
# vz.plot_signal(cfd.debug[1], cfd.debug[2])

# pdcc = pu.framing.PhaseDriftCFOCorrector(preamble, detection_threshold=t, detector_cls=pu.framing.AcquisitionFrameDetector)
# pdcc.process(sig)
# print(pdcc.get_estimate()/(2*np.pi))
# vz.plot_signal(pdcc.debug[0], preamble)

AttributeError: module 'py_utils' has no attribute 'framing'